In [37]:
import re
import pandas as pd

In [4]:
file = "HDFS.log"

In [26]:
hdfs = open(file, "r")

In [27]:
hdfs_list = hdfs.readlines()

In [101]:
#for i in range(1,100):
    #print(hdfs_list[i])

## Total Number of log entries

In [72]:
len(hdfs_list)

11175629

## Unique values for first 6 numerical digits

This looks like the dates of the events. Date doesn't seem like it will help with finding anomolies if we have many entries at each date.

In [73]:
first_six = []
for i in hdfs_list:
    match = re.findall('^[0-9]{6}', i)
    match = match[0]
    first_six.append(match)

In [82]:
unique_first_six = list(set(first_six))
print(unique_first_six)
len(first_six)

['081109', '081111', '081110']


11175629

## Unique values for second 6 numerical digits

It looks like there are tons of different values here, so this probably won't be significant in determining anomolies and we may not want it include it in our model. 

In [85]:
second_six = []
for i in hdfs_list:
    match = re.findall('^[0-9]{6} ([0-9]{6})', i)
    match = match[0]
    second_six.append(match)

In [87]:
unique_second_six = list(set(second_six))
print(len(unique_second_six))
len(second_six)

85053


11175629

## Unique values for third number code

There are a lot of different codes here. We will have to count through these to see if there are any that are rare. 

We may want to convert these to integers.

In [88]:
third_num = []
for i in hdfs_list:
    match = re.findall('^[0-9]{6} [0-9]{6} (\d*) ', i)
    match = match[0]
    third_num.append(match)

In [91]:
unique_third_num = list(set(third_num))
print(len(unique_third_num))
len(third_num)

27799


11175629

## Unique values for first text statement (4 capital letters)

In [68]:
caps = []
for i in hdfs_list:
    match = re.findall(' ([A-Z][A-Z][A-Z][A-Z]) ', i)
    match = match[0]
    caps.append(match)

In [93]:
unique_caps = list(set(caps))
print(unique_caps)
print(len(caps))

['INFO', 'WARN']
11175629


## Unique values for "dfs....:" statement

There are only 9 of these codes. These may be important for finding anomolies.

I think we could make a dataframe for each of these 9 codes, then we can find the specific information that are in each one. 

In [94]:
dfs = []
for i in hdfs_list:
    match = re.findall('dfs\.\S*:', i)
    match = match[0]
    dfs.append(match)

In [96]:
unique_dfs = list(set(dfs))
print(len(unique_dfs))
print(unique_dfs)
print(len(dfs))

9
['dfs.DataBlockScanner:', 'dfs.FSDataset:', 'dfs.DataNode$DataXceiver:', 'dfs.DataNode:', 'dfs.FSNamesystem:', 'dfs.DataNode$BlockReceiver:', 'dfs.PendingReplicationBlocks$PendingReplicationMonitor:', 'dfs.DataNode$DataTransfer:', 'dfs.DataNode$PacketResponder:']
11175629


In [97]:
hdfs.close()

## Labels information

It looks like the blocks are labelled, and each log entry contains at least one block. It looks like we will have to somehow find out which blocks are anomolies and which ones are normal. 

When do we determine if a block is normal or an anomoly?

Blocks must be in more than one event (there are a lot more events than anomoly labels), so are we finding events that are anomolies? Or blocks that are anomolies?

In [98]:
labels = pd.read_csv("anomaly_label.csv")

In [99]:
labels.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [100]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575061 entries, 0 to 575060
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   BlockId  575061 non-null  object
 1   Label    575061 non-null  object
dtypes: object(2)
memory usage: 8.8+ MB
